In [ ]:
# Import
import pandas as pd

In [188]:
# Test

True

In [216]:
measure_word_pleasantness('サカナハサカナ')

-5.942857142857143

# Flick

```python
{
    'origin': [0, 2],
    'direction': 'left'
}
```

In [ ]:
# Data

directions = ['none', 'left', 'up', 'right', 'down']
direction_movements = [[0, 0], [-2, 0], [0, -2], [2, 0], [0, 2]]

key_map = [
    ['ア', 'イ', 'ウ', 'エ', 'オ'],
    ['カ', 'キ', 'ク', 'ケ', 'コ'],
    ['サ', 'シ', 'ス', 'セ', 'ソ'],
    ['タ', 'チ', 'ツ', 'テ', 'ト'],
    ['ナ', 'ニ', 'ヌ', 'ネ', 'ノ'],
    ['ハ', 'ヒ', 'フ', 'ヘ', 'ホ'],
    ['マ', 'ミ', 'ム', 'メ', 'モ'],
    ['ヤ',   '', 'ユ',   '', 'ヨ'],
    ['ラ', 'リ', 'ル', 'レ', 'ロ'],
    ['゛',   '',   '', '゜',   ''],
    ['ワ', 'ヲ', 'ン', 'ー',   ''],
    ['、', '。', '？', '！',   '']
]

# Voicing
voiced_katakanas = [
    'ヴ',
    'ガ', 'ギ', 'グ', 'ゲ', 'ゴ',
    'ザ', 'ジ', 'ズ', 'ゼ', 'ゾ',
    'ダ', 'ヂ', 'ヅ', 'デ', 'ド',
    'バ', 'ビ', 'ブ', 'ベ', 'ボ'
]
voiceless_katakanas = [
    'ウ',
    'カ', 'キ', 'ク', 'ケ', 'コ',
    'サ', 'シ', 'ス', 'セ', 'ソ',
    'タ', 'チ', 'ツ', 'テ', 'ト',
    'ハ', 'ヒ', 'フ', 'ヘ', 'ホ'
]

# P
p_katakanas = [
    'パ', 'ピ', 'プ', 'ペ', 'ポ'
]
non_p_katakanas = [
    'ハ', 'ヒ', 'フ', 'ヘ', 'ホ'
]

# Small
small_katakanas = [
    'ァ', 'ィ', 'ゥ', 'ェ', 'ォ',
    'ッ',
    'ャ', 'ュ', 'ョ'
]
non_small_katakanas = [
    'ア', 'イ', 'ウ', 'エ', 'オ',
    'ツ',
    'ヤ', 'ユ', 'ヨ'
]

In [ ]:
# Utility

def add_positions(position1, position2):
    return [position1[0] + position2[0], position1[1] + position2[1]]

def subtract_positions(position1, position2):
    return [position1[0] - position2[0], position1[1] - position2[1]]

def direction_to_axis(direction):
    if direction == 'left' or direction == 'right':
        return 'x'
    elif direction == 'up' or direction == 'down':
        return 'y'
    else:
        return None

def is_same_axis(direction1, direction2):
    return direction_to_axis(direction1) == direction_to_axis(direction2)

In [ ]:
# Convert a string of katakanas to flicks

# 'ガ' -> 'カ゛' etc.
def decompose_katakanas(string):
    # Decompose voiced katakanas
    for voiced_and_voiceless in zip(voiced_katakanas, voiceless_katakanas):
        if voiced_and_voiceless[0] == 'ヴ':
            string = string.replace(voiced_and_voiceless[0], voiced_and_voiceless[1] + '゛゛')
        else:
            string = string.replace(voiced_and_voiceless[0], voiced_and_voiceless[1] + '゛')
    
    # Decompose P-katakanas
    for p_and_non_p in zip(p_katakanas, non_p_katakanas):
        string = string.replace(p_and_non_p[0], p_and_non_p[1] + '゜')
    
    # Decompose small katakanas
    for small_and_non_small in zip(small_katakanas, non_small_katakanas):
        string = string.replace(small_and_non_small[0], small_and_non_small[1] + '゛')
    
    return string

# 'ア' -> [{'origin': [0, 0], 'direction': 'none'}] etc.
def katakanas_to_flicks(decomposed_katakana_string):
    flicks = []
    
    for katakana in decomposed_katakana_string: # Loop through the katakanas of the passed string
        for key_index, katakanas_in_key in enumerate(key_map): # Loop through the keys to find the current katakana
            if katakana in katakanas_in_key: # If the current katakana is found in the current key
                flicks.append({
                    'origin': [key_index % 3, key_index // 3],
                    'direction': directions[katakanas_in_key.index(katakana)]
                })
                break
    
    return flicks

In [184]:
# Detect patterns in a sequence of directions

def detect_same_direction_sequence(directions, direction_to_detect):
    sequence_length_list = []
    current_sequence_length = 0
    
    for direction in directions:
        if direction == direction_to_detect:
            current_sequence_length += 1
        else: # End the sequence
            if current_sequence_length >= 3:
                sequence_length_list.append(current_sequence_length)
            current_sequence_length = 0
    if current_sequence_length >= 3:
        sequence_length_list.append(current_sequence_length)
    
    return sequence_length_list

def detect_alternate_directions_sequence(directions, direction1_to_detect, direction2_to_detect):
    sequence_length_list = []
    current_sequence_length = 0
    is_direction1_expected = True # Doesn't matter when current_sequence_length is 0
    
    for direction in directions:
        if current_sequence_length > 0:
            if (is_direction1_expected and direction == direction1_to_detect) \
                or (not is_direction1_expected and direction == direction2_to_detect):
                current_sequence_length += 1
                is_direction1_expected = not is_direction1_expected
            else: # End the sequence
                if current_sequence_length >= 3:
                    sequence_length_list.append(current_sequence_length)
                current_sequence_length = 0
        
        # If this direction is not in the previous sequence, start a new sequence
        if current_sequence_length == 0:
            if direction == direction1_to_detect:
                current_sequence_length += 1
                is_direction1_expected = False
            if direction == direction2_to_detect:
                current_sequence_length += 1
                is_direction1_expected = True
    if current_sequence_length >= 3:
        sequence_length_list.append(current_sequence_length)
    
    return sequence_length_list

In [218]:
# Measure pleasantness

def measure_shift_pleasantness(previous_position, next_position, previous_axis):
    delta = subtract_positions(next_position, previous_position)
    if previous_axis == 'x':
        delta[0] *= 0.5
    if previous_axis == 'y':
        delta[1] *= 0.5
    return -(delta[0] ** 2 + delta[1] ** 2) ** (1 / 2) / 7

# previous_flick is None if the current_flick is the first flick
def measure_single_input_pleasantness(previous_flick, current_flick):
    if previous_flick == None:
        shift_pleasantness = 0
        flick_pleasantness = -0.8 if current_flick['direction'] == 'none' else 0.5
    else:
        # Measure shift_pleasantness
        previous_end_position = add_positions(
            previous_flick['origin'],
            direction_movements[directions.index(previous_flick['direction'])]
        )
        shift_pleasantness = measure_shift_pleasantness(
            previous_end_position,
            current_flick['origin'],
            direction_to_axis(previous_flick['direction'])
        )
        
        # Measure flick_pleasantness
        if current_flick['direction'] == 'none':
            flick_pleasantness = -0.8
        else:
            if is_same_axis(previous_flick['direction'], current_flick['direction']):
                flick_pleasantness = 0.8
            else:
                flick_pleasantness = -0.7
            flick_pleasantness -= 0.2 if current_flick['direction'] == 'up' else 0
            flick_pleasantness -= 0.1 if current_flick['direction'] == 'left' else 0
        
    return shift_pleasantness + flick_pleasantness

def measure_flicks_pleasantness(flicks):
    total_pleasantness = 0
    
    # Individual pleasantness
    for index, flick in enumerate(flicks):
        previous_flick = None if index == 0 else flicks[index - 1]
        #print(measure_single_input_pleasantness(previous_flick, flick))
        total_pleasantness += measure_single_input_pleasantness(previous_flick, flick)
    
    # Pattern pleasantness
    flick_directions = [flick['direction'] for flick in flicks] # Different from directions defined above
    # Skip single 'none's
    #for i in reversed(range(len(flick_directions))):
    #    if (i == 0 and flick_directions[i] == 'none') \
    #        or (i != 0 and flick_directions[i] == 'none' and flick_directions[i - 1] != 'none'):
    #        del flick_directions[i]
    # Add the same direction sequence pleasantness
    for direction in directions[1:]:
        total_pleasantness += sum(
            [length ** 1.2 for length in detect_same_direction_sequence(
                flick_directions,
                direction
            )]
        ) / 2
    # Add the alternate directions sequence pleasantness
    for direction_pair in [['left', 'right'], ['up', 'down']]:
        total_pleasantness += sum(
            [length ** 1.2 for length in detect_alternate_directions_sequence(
                flick_directions,
                direction_pair[0],
                direction_pair[1]
            )]
        ) / 2
    
    return total_pleasantness

def measure_word_pleasantness(word):
    return measure_flicks_pleasantness(katakanas_to_flicks(decompose_katakanas(word)))

In [222]:
# Measure the pleasantness of the words in the dictionary

#FILENAME = 'ipadic'
FILENAME = 'hatena'
#FILENAME = 'practical'

df = pd.read_csv('dic/' + FILENAME + '.csv', encoding='utf-8', header=None)

for i, row in df.iterrows():
    if i % 10000 == 0:
        print(i)
    row.iloc[1] = measure_word_pleasantness(row.iloc[1])

sorted_df = df.sort_values(1, ascending=False)
print(sorted_df)
sorted_df.to_csv('dic/' + FILENAME + '_pleasantness.csv', encoding='utf-8', header=None, index=None)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
                                                0        1
320481                                 BaaaaaaaaQ  16.2667
291430                               ノーーーーーーーーーート  12.5816
381852                                  ムムムムムムムムム  12.5388
210747                              全国交通運輸労働組合総連合  12.5285
200922                               すもももももももものうち  12.3499
324035                                      PPPHG   12.199
48919                                        HHhH  12.1201
22683                                       EeePC  12.1151
324019                                   ピーピーエーピー  12.1021
324034                                       ＰＰＰＨ  12.0213
285589                                     日本航空高校  11.9142
32333